In [1]:
import warnings
warnings.filterwarnings('ignore')

#### IMPORTING NECESSARY MODULES

In [26]:
!pip install yfinance==0.2.43 gradio==4.42.0 gradio-client==1.3.0 plotly==5.24.0 tabulate==0.9.0 prettytable==3.11.0 tslearn==0.6.3 hmmlearn==0.3.2 pandas-ta==0.3.14b0 ta==0.11.0 pandas==2.0.3 numpy==1.24.4 matplotlib==3.7.5 matplotlib-inline==0.1.7

You should consider upgrading via the 'c:\users\nischay\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as  plt
import gradio as gr
import plotly.graph_objects as go
import plotly.io as pio
import datetime
import time
from tslearn.clustering import TimeSeriesKMeans
from hmmlearn import hmm
from dateutil.relativedelta import relativedelta
from tabulate import tabulate
from prettytable import PrettyTable
import math
import seaborn as sns
from tslearn.metrics import dtw
from sklearn.preprocessing import StandardScaler

 ### BACKTESTING ENGINE

In [3]:
class RiskManagement:
    def __init__(self):
        return

    '''This function is to set the stop-loss depending on entry price'''
    @staticmethod
    def set_stop_loss(ts, i):
        ts.sample = 1
        if ts.current_position == 1:
            ts.current_stop_loss_value = (1 - ts.stop_loss_percent*ts.sample) * (ts.current_trade_peak)
        elif ts.current_position == -1:
            ts.current_stop_loss_value = (1 + ts.exit_short_percent*ts.sample) * (ts.current_trade_peak)

    '''This function is used to update the stop-loss value, if the portfolio value increases (TRAILING STOP-LOSS)'''
    @staticmethod
    def update_stop_loss(ts, i):
        if ts.current_position == 1:
            if ts.holding * ts.close[i] > ts.current_trade_peak:
                ts.current_trade_peak = ts.holding * ts.close[i]
                RiskManagement.set_stop_loss(ts, i)
        elif ts.current_position == -1:
            if ts.capital - ts.holding * ts.close[i] < ts.current_trade_peak:
                ts.current_trade_peak = ts.capital - ts.holding * ts.close[i]
                RiskManagement.set_stop_loss(ts, i)

    '''This function is to set the take-profit depending on entry price'''
    @staticmethod
    def set_take_profit(ts, i):
        ts.sample=1
        if ts.current_position == 1:
            ts.take_profit_value = (1 + ts.take_profit_percent*ts.sample) * (ts.current_portfolio_value)
        elif ts.current_position == -1:
            ts.take_profit_value = (1 - ts.take_profit_percent*ts.sample) * (ts.current_portfolio_value)

class TradingStrategy_Compounding:

    '''This function initializes the class according to the data provided, it creates several variables for inside the class, as described'''
    def __init__(self, data):

        self.capital = 100000 # current capital
        self.data = data # data (OHLCV and signals)

        self.datetime = [] # datetime colmun
        self.low = data.low
        self.portfolio_value = [] # portfolio value at each index
        self.quantity = [] # holding at each index
        self.current_position = 0 # current position
        self.holding = 0 # current holding
        self.sample = 1
        self.current_portfolio_value = 0 # current portfolio value
        self.mul = 5
        self.current_stop_loss_value = 0 # stop-loss value
        self.current_trade_peak = 0 # for trailing stop-loss
        self.stop_loss_percent = 0.15# stop-loss percent/
        self.exit_short_percent = 100# exit condition fo/r short trade
        self.take_profit_value = 0 # temporary variable for the trade in action
        self.take_profit_percent = 0.35 # take-profit percen/t
        self.multipleir = 1
        self.stop_loss_count = [] # stop-loss count
        self.take_profit_count = [] # take-profit count

        self.entry = [] # entry index of
        self.exit = [] # exit index
        self.new_signals = [] # list of new signals
        self.close = data['close'] # close value of btc-usdt

        self.trade_type = [] # this is for trade type, according to entry type
        self.drawdown = [] # this is drawdown for each day
        self.benchmark_return = (((100000/self.close[0]) * self.close[len(self.data) - 1] - 100000)) # this is benchmark returns, according to buy & hold

        self.signals = data['signals'] # signals column according to strategy
        self.transaction_percentage = 0.00

        self.risk_free_rate = 0.05 # you can change it

        self.amount_in_trade = []

        self.trade_wise_returns = []
        self.PL_in_dollars = []

        self.amount_invested_in_trade = []

        if self.signals.empty:
            print("No signals generated, empty array encountered.")

    '''This function is used to start a long position on the equity'''
    def take_long_position(self,i):
        self.current_position = 1
        self.holding = self.capital / self.close[i] # as we buy the equity from all the capital we posses
        self.capital = 0
        self.new_signals.append(1)
        self.quantity.append(self.holding)
        self.current_portfolio_value = self.holding * self.close[i]
        self.portfolio_value.append(self.current_portfolio_value)
        self.entry.append(i)
        self.trade_type.append('long')
        self.amount_in_trade.append(self.current_portfolio_value)
        self.amount_invested_in_trade.append(self.current_portfolio_value)

    '''This function is used to start a short position on the equity'''
    def take_short_position(self,i):
        self.current_position = -1
        self.holding = self.capital / self.close[i]
        self.capital = 2 * self.capital
        self.new_signals.append(-1)
        self.quantity.append(self.holding)
        self.current_portfolio_value = self.capital - self.holding * self.close[i]
        self.portfolio_value.append(self.current_portfolio_value)
        self.entry.append(i)
        self.trade_type.append('short')
        self.amount_in_trade.append(self.current_portfolio_value)
        self.amount_invested_in_trade.append(self.current_portfolio_value)

    '''This function is called when we currently have no position, and do not intend to start either. So portfolio value will be same as capital'''
    def update_no_trade(self,i):
        self.current_position = 0
        self.new_signals.append(0)
        self.holding = 0
        self.portfolio_value.append(self.capital)
        self.quantity.append(0)

    '''This function is called when we are currently on long, and we don't want to exit the trade,so current holding and capital remain same'''
    def update_long_trade(self, i):
        self.current_position = 1
        self.quantity.append(self.holding)
        self.current_portfolio_value = self.holding * self.close[i]
        self.portfolio_value.append(self.current_portfolio_value)
        self.new_signals.append(0)

    '''This function is called when we are currently on short, and we don't want to exit the trade,so current holding and capital remain same'''
    def update_short_trade(self, i):
        self.current_position = -1
        self.quantity.append(self.holding)
        self.current_portfolio_value = self.capital - self.holding * self.close[i]
        self.portfolio_value.append(self.current_portfolio_value)
        self.new_signals.append(0)

    '''This function is called when we want to exit a long trade, so we will increase in hand capital in this case'''
    def close_long_trade(self, i):
        self.current_position = 0
        self.quantity.append(0)
        self.capital = self.close[i] * self.holding
        self.current_portfolio_value = self.capital
        self.portfolio_value.append(self.current_portfolio_value)
        self.new_signals.append(-1)
        self.exit.append(i)
        self.amount_in_trade.append(self.current_portfolio_value)

    '''This function is called when we want to exit a short trade, so we will increase in hand capital in this case'''
    def close_short_trade(self, i):
        self.current_position = 0
        self.capital = self.capital - self.close[i] * self.holding
        self.quantity.append(0)
        self.current_portfolio_value = self.capital
        self.portfolio_value.append(self.current_portfolio_value)
        self.new_signals.append(1)
        self.exit.append(i)
        self.amount_in_trade.append(self.current_portfolio_value)

    '''This function is called when we are currently on long/short position. It checks if we should exit the trade based on stop-loss and take-profit'''
    def check_exit_condition(self, i):
        if self.current_position == 1:
            temp_value = self.holding * self.close[i]
            if temp_value < self.current_stop_loss_value:
                self.stop_loss_count.append(i)
                return 1

            elif temp_value >= self.take_profit_value:
                self.take_profit_count.append(i)
                return 1

        elif self.current_position == -1:
            temp_value = self.capital - self.holding * self.close[i]
            if temp_value < self.current_stop_loss_value:
                self.stop_loss_count.append(i)
                return 1
            elif temp_value >= self.take_profit_value:
                self.take_profit_count.append(i)
                return 1
        return 0

    '''This is the function, which when called will analyse all the trades'''
    def compounding(self):

        x = len(self.data) - 1 # last trade will be dealt later
        for i in range(x):
            self.datetime.append(self.data.datetime[i])
            if self.capital < 0: # this possibility may arise in compounding approach
                print('capital wiped')

            if self.current_position == 0:

                if self.signals[i] == 0:
                    self.update_no_trade(i)
                elif self.signals[i] == 1:
                    self.take_long_position(i) # to start a new long position
                    self.current_trade_peak = self.current_portfolio_value
                    RiskManagement.set_take_profit(self, i)
                    RiskManagement.set_stop_loss(self, i)
                elif self.signals[i] == -1:
                    self.take_short_position(i) # to start a new short position
                    self.current_trade_peak = self.current_portfolio_value
                    RiskManagement.set_take_profit(self, i)
                    RiskManagement.set_stop_loss(self, i)

            elif self.current_position == 1:
                if self.signals[i] == 0 or self.signals[i] == 1:
                    if self.check_exit_condition(i) == 1:
                        self.close_long_trade(i) # to close a long position
                    else:
                        RiskManagement.update_stop_loss(self, i)
                        self.update_long_trade(i)
                else:
                    self.close_long_trade(i) # to close a long position

            elif self.current_position == -1:
                if self.signals[i] == 0 or self.signals[i] == -1:
                    if self.check_exit_condition(i) == 1:
                        self.close_short_trade(i) # to close a short position
                    else:
                        RiskManagement.update_stop_loss(self, i)
                        self.update_short_trade(i)
                else:
                    self.close_short_trade(i) # to close a short position

        # for the last trade
        self.datetime.append(self.data.datetime[x])

        if self.current_position == 1:
            self.close_long_trade(x)
        elif self.current_position == -1:
            self.close_short_trade(x)
        else:
            self.update_no_trade(i)

        '''''''''''''''''''''''''''''''''''''''''''''''Trade log completed'''''''''''''''''''''''''''''''''''''''''''''''
        # calculating remaining parameters

        self.trade_wise_duration = np.array(self.exit) - np.array(self.entry)
        self.trade_wise_profit = []
        self.trade_wise_loss = []

        for i in range(len(self.entry)):
            current_trade_return = 100*((self.portfolio_value[self.exit[i]]/self.portfolio_value[self.entry[i]])-1)
            self.PL_in_dollars.append(self.portfolio_value[self.exit[i]]-self.portfolio_value[self.entry[i]])
            self.trade_wise_returns.append(current_trade_return)
            if current_trade_return >= 0:
                self.trade_wise_profit.append(current_trade_return)
            else:
                self.trade_wise_loss.append(current_trade_return)

        self.gross_profit = np.sum(self.PL_in_dollars)
        self.calculate_transaction_cost()
        self.net_profit = self.gross_profit - self.transaction_cost
        self.returns = self.net_profit / 100000 * 100

        metricstr = Compounding_Results.print_parameters(self)
        return Compounding_Results.create_strategy_dataframes(self) , Compounding_Results.create_trade_wise_dataframe(self) , Compounding_Results.create_every_day_dataframe(self), None, "".join([str(i[0])+ " : " +str(i[1])+"\n" for i in metricstr])

    def calculate_transaction_cost(self):
        self.transaction_cost = 0
        for i in range(len(self.entry)):
            self.transaction_cost =0
class Compounding_Results:
    def _init_(self):
        return

    '''After backtesting is complete, this function generated a dataframe which is the final one, after take-profit and stop-loss is implemented'''
    @staticmethod
    def create_strategy_dataframes(ts):
        to_submit = pd.DataFrame(columns=['datetime'])
        to_submit['datetime'] = ts.datetime
        to_submit['open'] = ts.data.open
        to_submit['high'] = ts.data.high
        to_submit['low'] = ts.data.low
        to_submit['close'] = ts.data.close
        to_submit['volume'] = ts.data.volume
        to_submit['signals'] = ts.new_signals
        return to_submit

    '''This function generated a trade-log for our strategy, giving returns in each trade'''
    @staticmethod
    def create_trade_wise_dataframe(ts):
        trade_wise = pd.DataFrame(columns=['entry', 'exit'])
        trade_wise['entry'] = ts.entry
        trade_wise['exit'] = ts.exit
        trade_wise['duration'] = ts.trade_wise_duration
        trade_wise['trade type'] = ts.trade_type
        trade_wise['returns'] = ts.trade_wise_returns
        return trade_wise

    '''This function creates and everyday log of our strategy, to analyze the portfolio value and drawdown for each day'''
    @staticmethod
    def create_every_day_dataframe(ts):
        every_day = pd.DataFrame(columns=['datetime', 'portfolio value', 'quantity'])
        every_day['datetime'] = ts.datetime
        every_day['quantity'] = ts.quantity
        every_day['portfolio value'] = ts.portfolio_value
        every_day['daily_return'] = every_day['portfolio value'].pct_change()
        every_day['signals'] = every_day['portfolio value'].pct_change()
        every_day['signals'] = ts.new_signals
        every_day['Close'] = ts.data.close # not needed
        return every_day

    '''This function is used to print the necassary parameters, useful for analyzing our strategy'''
    @staticmethod
    def print_parameters(ts):
        # Plotting portfolio value
        try:
            maxd=0
            maxp=0
            for i in ts.portfolio_value:
                maxp=max(maxp, i)
                maxd= max((maxp-i)/maxp * 100, maxd)

            
            ret = np.diff(ts.portfolio_value) / np.array(ts.portfolio_value[:-1])
            sharpe = np.mean(ret) / np.std(ret) * np.sqrt(252)
                
            plt.plot(ts.portfolio_value)
            dat = [
                ["Number of closed trades", len(ts.entry)],
                ["Winning trades", len(ts.trade_wise_profit)],
                ["Losing trades", len(ts.trade_wise_loss)],
                ["Benchmark returns", round(ts.benchmark_return, 2)],
                ["Win rate", (round((len(ts.trade_wise_profit) / len(ts.entry)) * 100, 2)) if len(ts.entry) != 0 else 0],
                ["Largest win", round(np.max(ts.trade_wise_profit), 2)],
                ["Average win", round(np.mean(ts.trade_wise_profit), 2)],
                ["Largest loss", round(np.min(ts.trade_wise_loss), 2)],
                ["Average loss", round(np.mean(ts.trade_wise_loss), 2)],
                ["Maximum holding time", round(np.max(ts.trade_wise_duration), 2)],
                ["Average holding period", round(np.mean(ts.trade_wise_duration), 2)],
                #["Gross Profit", round(ts.gross_profit, 2)],
                ["Net Profit", round(ts.net_profit, 2)],
                ["Returns", f"{round(ts.returns, 2)} %"],
                ["Sharpe", f"{sharpe}"],
                ["Drawdown", f"{maxd}"]
            ]
    
            # Calculate the maximum length of the second column (Value column)
            max_len_metric = max(len(row[0]) for row in dat)
    
            # Pad both columns based on the maximum length
            padded_dat = [[row[0].ljust(max_len_metric), str(row[1]).rjust(10)] for row in dat]
    
            # Specify the table format, for example, "grid", "pipe", "html", "latex", etc.
            table_format = "grid"
    
            # Print the table
            print(tabulate(padded_dat, headers=["Metric", "Value"], tablefmt=table_format))
        except Exception as e:
            print("ERROR IN PRINT_PARAMETERS", e)
            dat = [[str(sharpe), str(e)]]
        return dat
    

### FINDING ICHIMOKU PARAMETERS FOR VOLATILITY CALCULATION

In [4]:
def ichimoku(data_comb):
    high_9 = data_comb['High'].rolling(window=9).max()
    low_9 = data_comb['Low'].rolling(window=9).min()
    tenkan_sen = (high_9 + low_9) / 2
    
    high_26 = data_comb['High'].rolling(window=26).max()
    low_26 = data_comb['Low'].rolling(window=26).min()
    kijun_sen = (high_26 + low_26) / 2
    
    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)
    senkou_span_b = ((high_52 := data_comb['High'].rolling(window=52).max()) + (low_52 := data_comb['Low'].rolling(window=52).min())) / 2
    senkou_span_b = senkou_span_b.shift(26)
    
    chikou_span = data_comb['Close'].shift(-26)
    
    ema_12 = data_comb['Close'].ewm(span=12, adjust=False).mean()
    ema_26 = data_comb['Close'].ewm(span=26, adjust=False).mean()
    macd = ema_12 - ema_26
    macd_signal = macd.ewm(span=9, adjust=False).mean()
    
    bullish = (data_comb['Close'] > senkou_span_a) & (data_comb['Close'] > senkou_span_b) & (tenkan_sen > kijun_sen) & (macd > macd_signal)
    bearish = (data_comb['Close'] < senkou_span_a) & (data_comb['Close'] < senkou_span_b) & (tenkan_sen < kijun_sen) & (macd < macd_signal)
    sideways = ~(bullish | bearish)
    
    data_comb['moment'] = np.where(bullish, 1, np.where(bearish, 0, 2))
    
    cloud_trace_a = go.Scatter(x=data_comb.index, y=senkou_span_a, mode='lines', name='Senkou Span A', line=dict(color='lightgreen'))
    cloud_trace_b = go.Scatter(x=data_comb.index, y=senkou_span_b, mode='lines', name='Senkou Span B', line=dict(color='lightcoral'), fill='tonexty')
    
    bullish_trace = go.Scatter(x=data_comb.index, y=np.where(bullish, data_comb['Close'], np.nan), mode='lines', name='Bullish', line=dict(color='green'), opacity=0.5)
    bearish_trace = go.Scatter(x=data_comb.index, y=np.where(bearish, data_comb['Close'], np.nan), mode='lines', name='Bearish', line=dict(color='red'), opacity=0.5)
    sideways_trace = go.Scatter(x=data_comb.index, y=np.where(sideways, data_comb['Close'], np.nan), mode='lines', name='Sideways', line=dict(color='orange'), opacity=0.5)
    
    ichi = [cloud_trace_a, cloud_trace_b, bullish_trace, bearish_trace, sideways_trace]
    return ichi

HELPER FUNCTION TO FETCH AND PREPROCESS DATA

In [5]:
def gen_newdata(data):
    p = data.copy()
    p['return'] = p['Close'].pct_change()
    p['momentum'] = p['Close'] - p['Close'].shift(10)
    p['short_ma'] = p['Close'].rolling(window=20).mean()
    p['long_ma'] = p['Close'].rolling(window=50).mean()
    p['trend'] = p['short_ma'] - p['long_ma']
    p['volatility'] = p['Close'].rolling(window=26).std()
    p['rsi'] = 100 - (100 / (1 + p['Close'].diff(1).apply(lambda x: max(x, 0)).rolling(window=14).mean() / p['Close'].diff(1).apply(lambda x: abs(min(x, 0))).rolling(window=14).mean()))
    p['TR'] = np.maximum(p['High'] - p['Low'],
                     np.maximum(abs(p['High'] - p['Close'].shift(1)),
                                abs(p['Low'] - p['Close'].shift(1))))
    p['ATR'] = p['TR'].rolling(window=14).mean()
    p['atr']=p['TR'].rolling(window=14).mean()
    p['upper_wick'] = p['High'] - p[['Open', 'Close']].max(axis=1)
    p['lower_wick'] = p[['Open', 'Close']].min(axis=1) - p['Low']
    p['wick_to_body'] = (p['upper_wick'] + p['lower_wick']) / (p['High']-p['Low'])
    p['volume_spike'] = np.where(p['Volume'] > 1.5 * p['Volume'].rolling(window=20).mean(),1,0)
    return p

### FETCH DATA USING YFINANCE API

In [6]:
def get_preprocessed_data_back(ticker, sdate, edate):

    date_str = sdate
    date_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d")
    new_date_obj = date_obj - relativedelta(years=1)
    #import_date = new_date_obj.strftime("%Y-%m-%d")
    import_date= "2017-01-01"

    if(new_date_obj<datetime.datetime.strptime("2017-01-01", "%Y-%m-%d")):
        print("Error please enter date after or equal to 2018-01-01 as 1 year past data is being used")
        return -1, -1
    data_train = yf.download(ticker, import_date, sdate) # Open High Close Low Volume
    data=yf.download(ticker,sdate,edate)
    
    vix_symbol = "^VIX"
    
    data_train.columns = data_train.columns.str.capitalize()
    data.columns = data.columns.str.capitalize()
    vix_data_t = yf.download(vix_symbol, import_date, sdate)
    vix_data = yf.download(vix_symbol, sdate, edate)
    data_train['VIX']=vix_data_t['Close']
    data['VIX']=vix_data['Close']

    data=gen_newdata(data)
    data_train=gen_newdata(data_train)
    data = data.dropna()
    data_train = data_train.dropna()
    return data_train, data

### ICHIMOKU CLOUDS

In [7]:
def ichimoku_clustering_analysis(df, n_clusters=3, column_mapping=None, ichimoku_weight=0.8, clustering_weight=0.2):
    # Ensure the column mapping is provided
    if column_mapping is None:
        column_mapping = {
            'High': 'High',
            'Low': 'Low',
            'Close': 'Close'
        }

    # Compute Ichimoku components
    high_9 = df[column_mapping['High']].rolling(window=9).max()
    low_9 = df[column_mapping['Low']].rolling(window=9).min()
    tenkan_sen = (high_9 + low_9) / 2

    high_26 = df[column_mapping['High']].rolling(window=26).max()
    low_26 = df[column_mapping['Low']].rolling(window=26).min()
    kijun_sen = (high_26 + low_26) / 2

    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)
    high_52 = df[column_mapping['High']].rolling(window=52).max()
    low_52 = df[column_mapping['Low']].rolling(window=52).min()
    senkou_span_b = ((high_52 + low_52) / 2).shift(26)

    ema_12 = df[column_mapping['Close']].ewm(span=12, adjust=False).mean()
    ema_26 = df[column_mapping['Close']].ewm(span=26, adjust=False).mean()
    macd = ema_12 - ema_26
    macd_signal = macd.ewm(span=9, adjust=False).mean()

    
    bullish = (df[column_mapping['Close']] > senkou_span_a) & (df[column_mapping['Close']] > senkou_span_b) & (tenkan_sen > kijun_sen) & (macd > macd_signal)
    bearish = (df[column_mapping['Close']] < senkou_span_a) & (df[column_mapping['Close']] < senkou_span_b) & (tenkan_sen < kijun_sen) & (macd < macd_signal)
    sideways = ~(bullish | bearish)

    trend_classification_ichimoku = np.where(bullish, 1, np.where(bearish, 0, 2))

    
    p = df.copy()
    """
    p['return'] = p['Close'].pct_change()  # Price returns
    p['momentum'] = p['Close'] - p['Close'].shift(10)  # Momentum over 10 days
    p['short_ma'] = p['Close'].rolling(window=20).mean()  # 20-day moving average
    p['long_ma'] = p['Close'].rolling(window=50).mean()  # 50-day moving average
    p['trend'] = p['short_ma'] - p['long_ma']  # Trend difference
    p['volatility'] = p['Close'].rolling(window=26).std()  # Volatility
    p['rsi'] = 100 - (100 / (1 + p['Close'].diff(1).apply(lambda x: max(x, 0)).rolling(window=14).mean() / p['Close'].diff(1).apply(lambda x: abs(min(x, 0))).rolling(window=14).mean()))  # RSI
    p['VIX']=vix_data['Close']
    p['TR'] = np.maximum(p['High'] - p['Low'],
                     np.maximum(abs(p['High'] - p['Close'].shift(1)),
                                abs(p['Low'] - p['Close'].shift(1))))
    p['ATR'] = p['TR'].rolling(window=14).mean()
    p['upper_wick'] = p['High'] - p[['Open', 'Close']].max(axis=1)
    p['lower_wick'] = p[['Open', 'Close']].min(axis=1) - p['Low']
    p['wick_to_body'] = (p['upper_wick'] + p['lower_wick']) / (p['High']-p['Low'])
    p['volume_spike'] = np.where(p['Volume'] > 1.5 * p['Volume'].rolling(window=20).mean(),1,0)
    """

    # FEATURES BEING USED FOR CLUSTERING 6x
    features = np.column_stack([df['VIX'].fillna(0), df['wick_to_body'].fillna(0), df['trend'].fillna(0), df['ATR'].fillna(0), df['rsi'].fillna(50),df['volume_spike'].fillna(df['volume_spike'].mean()) ])

    features = np.array(features, dtype=np.float64)

    features = np.nan_to_num(features)

    
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    # features_scaled = scaler.fit_transform(features)
    features_scaled = (features)

    # TimeSeriesKMeans Clustering
    model = TimeSeriesKMeans(n_clusters=3, metric="dtw", max_iter=20, n_init=5)
    model.fit(features_scaled)
    p['trend_cluster'] = model.labels_

    # Assign clusters to different trends
    p_1 = p[p['trend_cluster'] == 0]
    p_2 = p[p['trend_cluster'] == 1]
    p_3 = p[p['trend_cluster'] == 2]
    #p_4 = p[p['trend_cluster'] == 3]
    #p_5 = p[p['trend_cluster'] == 4]

    # Plotting the results
    plt.figure(figsize=(20,15))
    plt.scatter(p_1.index.values, p_1['Close'], color='red', label='1')
    plt.scatter(p_2.index.values, p_2['Close'], color='green', label='2')
    plt.scatter(p_3.index.values, p_3['Close'], color='yellow', label='3')
    #plt.scatter(p_4.index.values, p_4['Close'], color='blue', label='4')
    #plt.scatter(p_5.index.values, p_5['Close'], color='black', label='5')


     # Time Series Clustering
    model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", verbose=False, random_state=42)
    clusters = model.fit_predict(features)

    x = df.index 
    y = clusters

    # Now plotting
    plt.scatter(x, y, c=y, cmap='viridis') 
    plt.title('Clustering of Reliance Stock Price into Bearish, Bullish, and Sideways Trends')
    plt.xlabel('Date')
    plt.ylabel('Cluster Labels')
    plt.legend()
    plt.show()
    
    # Calculating the weighted average trend classification (ichi+dtw)
    trend_classification = (ichimoku_weight * trend_classification_ichimoku + clustering_weight * clusters).round().astype(int)
    
    barycenters = model.cluster_centers_
    return clusters, barycenters,scaler


### INTEGRATING THE CLUSTERING ALGORITHM

In [8]:
def segment_and_majority_vote(arr, segment_length=20, threshold=0.7):

    segmented_arr = np.copy(arr)

    for i in range(0, len(arr), segment_length):
        # Extract the current segment (window_len sized)
        segment = arr[i:i + segment_length]

        # Find the majority group in the segment
        values, counts = np.unique(segment, return_counts=True)
        max_count = np.max(counts)
        majority_value = values[np.argmax(counts)]

        # Check if the majority value exceeds the threshold
        if max_count / len(segment) >= threshold:
            # Set all elements in the segment to the majority value
            segmented_arr[i:i + segment_length] = majority_value

    return segmented_arr

In [9]:
def trendpart(data,start_date,end_date):

    trend,centers,scaler=ichimoku_clustering_analysis(data, n_clusters=3, column_mapping=None, ichimoku_weight=0.5, clustering_weight=0.5)


    final_trend=segment_and_majority_vote(trend)
    data['cluster']=final_trend
    
    return data,centers,scaler

### STRATEGY 1

In [10]:
def strat1(data):
    df=data.copy()
    def identify_support_resistance(df, window=7):
        df['rolling_min'] = df['Low'].rolling(window=window).min()
        df['rolling_max'] = df['High'].rolling(window=window).max()
        df['support'] = df['rolling_min']
        df['resistance'] = df['rolling_max']
        return df

# Function to generate buy/sell signals based on support and resistance levels
    def generate_signals(df):
        df['signals'] = 0  # Default no position
        import numpy as np
        # Buy when price is near support, Sell when price is near resistance
        df['signals'] = np.where(df['Close'] <= df['support'], 1, df['signals'])  # Buy
        df['signals'] = np.where(df['Close'] >= df['resistance'], -1, df['signals'])  # Sell
        
        return df
    def heikin_Ashi(data):
        data['Close_Ha'] = (data['Open'] + data['High'] + data['Close'] + data['Low']) / 4
        data['Open_Ha'] = (data['Open'].shift(1) + data['Close'].shift(1)) / 2
        data['High_Ha'] = data[['High', 'Close_Ha', 'Open_Ha']].max(axis=1)
        data['Low_Ha'] = data[['Low', 'Close_Ha', 'Open_Ha']].min(axis=1)
        return data
    
    # Technical Indicators
    def RSI(df, period=14):
        delta = df['Close_Ha'].diff(1)
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window=period).mean()
        avg_loss = loss.rolling(window=period).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi
    
    def calculate_sma(data, window):
        return data.rolling(window=window).mean()
    
    def calculate_ema(data, window):
        return data.ewm(span=window, adjust=False).mean()
    
    def OBV(df):
        OBV = [0]
        for i in range(1, len(df)):
            if df['Close_Ha'][i] > df['Close_Ha'][i-1]:
                OBV.append(OBV[-1] + df['Volume'][i])
            elif df['Close_Ha'][i] < df['Close_Ha'][i-1]:
                OBV.append(OBV[-1] - df['Volume'][i])
            else:
                OBV.append(OBV[-1])
        return OBV

    data = heikin_Ashi(data)
    data['RSI'] = RSI(data)
    data['OBV'] = OBV(data)
    data['SMA'] = calculate_sma(data['Close_Ha'], 5)
    data['EMA'] = calculate_ema(data['Close_Ha'], 5)
    def strategy(data):
        signals = []
        for i in range(len(data)):
            if(data['RSI'].iloc[i]<=39 and data['Close'].iloc[i]<=data['SMA'].iloc[i] and data['OBV'].iloc[i]>=data['OBV'].iloc[i-1]):
                signals.append(1)
            elif(data['RSI'].iloc[i]>=69 and data['Close'].iloc[i]>=data['SMA'].iloc[i] and data['OBV'].iloc[i]<=data['OBV'].iloc[i-1]):
                signals.append(-1)
            else:
                signals.append(0)
        return signals
    #df = identify_support_resistance(df, window=10)

    # Generate buy/sell signals
    df = generate_signals(df)

    data['signals']=strategy(data)
    for i in range(len(data['signals'])):
        if(data['signals'].iloc[i]==0 and df['signals'].iloc[i]==1):
            data['signals'].iloc[i]=1
        elif(data['signals'].iloc[i]==0 and df['signals'].iloc[i]==-1):
            data['signals'].iloc[i]=-1
    

    #data['signals']=position
    
    
    return data


### STRATEGY 2

In [11]:
def strat2(data):
    df=data
    df.index = pd.to_datetime(df.index, utc=False)

    def calculate_huma(df, short_window=12, long_window=26, signal_window=9):
        df['hma_short'] = df['Close'].ewm(span=short_window, adjust=False).mean()
        df['hma_long'] = df['Close'].ewm(span=long_window, adjust=False).mean()
        df['huma'] = df['hma_short'] - df['hma_long']
        df['huma_line'] = df['huma'].ewm(span=signal_window, adjust=False).mean()
        return df
    
    df = calculate_huma(df)

# Calculate Chaikin Volatility
    def calculate_chaikin_volatility(df, ema_period=14):
        high_low_diff = df['High'] - df['Low']
        ema_diff = high_low_diff.ewm(span=ema_period, adjust=False).mean()
        chaikin_volatility = (ema_diff.diff() / ema_diff.shift(1)) * 100
        df['Chaikin_Volatility'] = chaikin_volatility
        return df

    df = calculate_chaikin_volatility(df)

# Calculate MFI (Money Flow Index)
    def calculate_mfi(df, period):
        tp = (df['High'] + df['Low'] + df['Close']) / 3
        mf = tp * df['Volume']
        pos_mf = mf.where(tp > tp.shift(1), 0).rolling(window=period).sum()
        neg_mf = mf.where(tp < tp.shift(1), 0).rolling(window=period).sum()
        mf_ratio = pos_mf / neg_mf
        return 100 - (100 / (1 + mf_ratio))

    def dynamic_mfi(df):
        # Calculate both short-term and long-term MFI
        df['MFI_Short'] = calculate_mfi(df, period=9)
        df['MFI_Long'] = calculate_mfi(df, period=14)
    
        # Determine which MFI to use based on Chaikin Volatility
        mean_volatility = df['Chaikin_Volatility'].mean()
        df['MFI'] = np.where(df['Chaikin_Volatility'] > mean_volatility, df['MFI_Short'], df['MFI_Long'])
    
        return df

    def calculate_ibs(df):
    
        ibs = (df['Close'] - df['Low']) / (df['High'] - df['Low'])
        return ibs

    df['IBS'] = calculate_ibs(df)
    
    df = dynamic_mfi(df)

    # Calculate ADX (Average Directional Index)
    def calculate_adx(df, period=14):
        df['TR'] = np.maximum(df['High'] - df['Low'],
                              np.maximum(abs(df['High'] - df['Close'].shift(1)),
                                         abs(df['Low'] - df['Close'].shift(1))))
        df['+DM'] = np.where((df['High'] - df['High'].shift(1)) > (df['Low'].shift(1) - df['Low']),
                             np.maximum(df['High'] - df['High'].shift(1), 0), 0)
        df['-DM'] = np.where((df['Low'].shift(1) - df['Low']) > (df['High'] - df['High'].shift(1)),
                             np.maximum(df['Low'].shift(1) - df['Low'], 0), 0)
        df['TR_smooth'] = df['TR'].rolling(window=period).mean()
        df['+DI'] = 100 * (df['+DM'].rolling(window=period).mean() / df['TR_smooth'])
        df['-DI'] = 100 * (df['-DM'].rolling(window=period).mean() / df['TR_smooth'])
        df['DX'] = 100 * abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI'])
        df['ADX'] = df['DX'].rolling(window=period).mean()
        return df
    
    df = calculate_adx(df)

    # Generate trade signals based on MACD, MFI, and ADX
    df['Buy_Signal'] = np.where((df['huma'] < df['huma_line']) & (df['MFI'] < 30) & (df['ADX'] > 25), 1, 0)
    df['Sell_Signal'] = np.where((df['huma'] > df['huma_line']) & (df['MFI'] > 70) & (df['ADX'] > 25), -1, 0)
    
    # Combine signals
    df['signals'] = df['Buy_Signal'] + df['Sell_Signal']
    def generate_signals(df):
        df['signals'] = 0  # Default no position
    
        # Buy when price is near support, Sell when price is near resistance
        df['signals'] = np.where(df['Close'] <= df['support'], 1, df['signals'])  # Buy
        df['signals'] = np.where(df['Close'] >= df['resistance'], -1, df['signals'])  # Sell
        
        return df
    df_cpy=df.copy()
    df_cpy=generate_signals(df_cpy)
    for i in range(len(df['Close'])):
        if(df['signals'].iloc[i]==0 and df_cpy['signals'].iloc[i]==1):
            df['signals'].iloc[i]=1
        if(df['signals'].iloc[i]==0 and df_cpy['signals'].iloc[i]==-1):
            df['signals'].iloc[i]=-1
    

    return df

### STRATEGY 3

In [12]:
def strat3(data):
    ## Signals based on cloud width
    data_cpy=data.copy()
    def Heiken_Ashi(df):
        df['HA_close']=(df['Open']+ df['High']+ df['Low']+ df['Close'])/4
        df['HA_open']=(df['Open']+df['Close'])/2
    
        for i in range(1, len(df)):
            df['HA_open'][i]=(df['HA_open'][i-1]+df['HA_close'][i-1])/2
            df['HA_high']=df[['HA_open','HA_close','High']].max(axis=1)
            df['HA_low']=df[['HA_open','HA_close','Low']].min(axis=1)
            df['Close']=df['HA_close']
            df['Open']=df['HA_open']
            df['High']=df['HA_high']
            df['Low']=df['HA_low']
        return df
    data_cpy=Heiken_Ashi(data_cpy)
    from numpy import nan as npnan
    from numpy import log as npLog
    from numpy import power as npPower
    from numpy import sqrt as npSqrt
    from numpy import zeros_like as npZeroslike
    from pandas import Series
    from pandas_ta.utils import get_offset, verify_series
    from numpy import average as npAverage
    def jma(close, length=None, phase=None, offset=None, **kwargs):
        """Indicator: Jurik Moving Average (JMA)"""
        #This is a proprietary denoising algo that I searched for from internet
        # Validate Arguments
        _length = int(length) if length and length > 0 else 7
        phase = float(phase) if phase and phase != 0 else 0
        close = verify_series(close, _length)
        offset = get_offset(offset)
        if close is None: return
    
        # Define base variables
        jma = npZeroslike(close)
        volty = npZeroslike(close)
        v_sum = npZeroslike(close)
    
        kv = det0 = det1 = ma2 = 0.0
        jma[0] = ma1 = uBand = lBand = close[0]
    
        # Static variables
        sum_length = 10
        length = 0.5 * (_length - 1)
        pr = 0.5 if phase < -100 else 2.5 if phase > 100 else 1.5 + phase * 0.01
        length1 = max((npLog(npSqrt(length)) / npLog(2.0)) + 2.0, 0)
        pow1 = max(length1 - 2.0, 0.5)
        length2 = length1 * npSqrt(length)
        bet = length2 / (length2 + 1)
        beta = 0.45 * (_length - 1) / (0.45 * (_length - 1) + 2.0)
    
        m = close.shape[0]
        for i in range(1, m):
            price = close[i]
    
            # Price volatility
            del1 = price - uBand
            del2 = price - lBand
            volty[i] = max(abs(del1),abs(del2)) if abs(del1)!=abs(del2) else 0
    
            # Relative price volatility factor
            v_sum[i] = v_sum[i - 1] + (volty[i] - volty[max(i - sum_length, 0)]) / sum_length
            avg_volty = npAverage(v_sum[max(i - 65, 0):i + 1])
            d_volty = 0 if avg_volty ==0 else volty[i] / avg_volty
            r_volty = max(1.0, min(npPower(length1, 1 / pow1), d_volty))
    
            # Jurik volatility bands
            pow2 = npPower(r_volty, pow1)
            kv = npPower(bet, npSqrt(pow2))
            uBand = price if (del1 > 0) else price - (kv * del1)
            lBand = price if (del2 < 0) else price - (kv * del2)
    
            # Jurik Dynamic Factor
            power = npPower(r_volty, pow1)
            alpha = npPower(beta, power)
    
            # 1st stage - prelimimary smoothing by adaptive EMA
            ma1 = ((1 - alpha) * price) + (alpha * ma1)
    
            # 2nd stage - one more prelimimary smoothing by Kalman filter
            det0 = ((price - ma1) * (1 - beta)) + (beta * det0)
            ma2 = ma1 + pr * det0
    
            # 3rd stage - final smoothing by unique Jurik adaptive filter
            det1 = ((ma2 - jma[i - 1]) * (1 - alpha) * (1 - alpha)) + (alpha * alpha * det1)
            jma[i] = jma[i-1] + det1
    
        # Remove initial lookback data and convert to pandas frame
        jma[0:_length - 1] = npnan
        jma = Series(jma, index=close.index)
    
        # Offset
        if offset != 0:
            jma = jma.shift(offset)
    
        # Handle fills
        if "fillna" in kwargs:
            jma.fillna(kwargs["fillna"], inplace=True)
        if "fill_method" in kwargs:
            jma.fillna(method=kwargs["fill_method"], inplace=True)
    
        # Name & Category
        jma.name = f"JMA_{_length}_{phase}"
        jma.category = "overlap"
    
        return jma
    
    data_cpy['jma']=jma(data_cpy['Close'],7)
    def calculate_weighted_avg(data, span=5):
    
        data['Upward_price_movement'] = data['Close'] - data['Low']
        data['Candle_length'] = data['High'] - data['Low']
        data['IBS'] = data['Upward_price_movement'] / data['Candle_length']
        data['weight_avg'] = data['IBS'].ewm(span=span, adjust=False).mean()
        return data
    
    data_cpy = calculate_weighted_avg(data_cpy)
    def hma(data,period):
         data['sma_5'] = data['Close'].ewm(span=5,adjust=False).mean()
         wma_1 = data['Close'].rolling(period//2).apply(lambda x: \
         np.sum(x * np.arange(1, period//2+1)) / np.sum(np.arange(1, period//2+1)), raw=True)
         wma_2 = data['Close'].rolling(period).apply(lambda x: \
         np.sum(x * np.arange(1, period+1)) / np.sum(np.arange(1, period+1)), raw=True)
         diff = 2 * wma_1 - wma_2
         hma = diff.rolling(int(np.sqrt(period))).mean()
         data['hma'] = hma
         return data


    data_cpy = hma(data_cpy, 15)
    def atr(data):
      period = 20
    
      data['TR'] = 0
    
      for i in range(1, len(data)):
          high_low_range = abs(data['High'][i] - data['Low'][i])  # High-Low Range
          high_prev_close_range = abs(data['High'][i] - data['Close'][i - 1])  # High-Previous Close Range
          low_prev_close_range = abs(data['Low'][i] - data['Close'][i - 1])  # Low-Previous Close Range
    
          # Assign the maximum value of the three ranges to the 'TR' column for the current row
          data['TR'][i] = max(high_low_range, high_prev_close_range, low_prev_close_range)
      data['atr'] = 0
      data['atr'] = data['TR'].rolling(window=period).mean()
      data['signal_atr']=data['atr'].ewm(span=25).mean()  #ewm check
    
      data['slope_atr']=(data['signal_atr']-data['signal_atr'].shift(10))/10
    
      return data
    
    data_cpy=atr(data_cpy)
    data=atr(data)
    def Elder_Ray(data,ema_period=13):
        data['EMA'] = data['Close'].ewm(span=ema_period, adjust=False).mean()
    
        # Calculate Bull Power
        data['Bull Power'] = data['High'] - data['EMA']
    
        # Calculate Bear Power
        data['Bear Power'] = data['Low'] - data['EMA']
        return data
    
    data_cpy=Elder_Ray(data_cpy)
    def self_strat(data):
      #WHEN HIGH STANDARD DEVIATION TAKE TRADES
      a=np.where(data['Close'].rolling(window=5).std()>0.7,1,0)
      #WHEN LOW STANDARD DEVIATION TAKE TRADES
      b=np.where(data['Close'].rolling(window=5).std()<0.3,-1,0)
      data['self_signals']=a+b
      x=data['Close'].rolling(window=10).std()
      y=x.ewm(span=5).mean()
      z=x.rolling(window=5).mean()
      data['self_signals_2']=np.where(y>z,1,-1)
      data['sma_10']=data['Close'].rolling(window=10).mean()
      data['ema_10']=data['Close'].ewm(span=10,adjust=False).mean()
      data['sma_5']=data['Close'].rolling(window=5).mean()
      data['ewm_5']=data['Close'].ewm(span=5,adjust=False).mean()
      data['h_l']=data['High']-data['Low']
      z=data['h_l'].rolling(window=5).std()
      data['self_signals_3']=np.where(z>0.3,1,-1)
      return data
    data_cpy=self_strat(data_cpy)
    def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
        data['EMA12'] = data['Close'].ewm(span=short_window, adjust=False).mean()
        data['EMA26'] = data['Close'].ewm(span=long_window, adjust=False).mean()
        data['MACD'] = data['EMA12'] - data['EMA26']
        data['Signal'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
        return data
    
    
    data_cpy['macd_signals'] = 0
    data_cpy=calculate_macd(data_cpy)
    # Generate signals based on the MACD line and Signal line crossovers
    data_cpy['macd_signals'][1:] = np.where(data_cpy['MACD'][1:] > data_cpy['Signal'][1:], 1, 0)
    data_cpy['macd_signals'][1:] = np.where(data_cpy['MACD'][1:] < data_cpy['Signal'][1:], -1, data_cpy['macd_signals'][1:])
    from ta.trend import ADXIndicator
    def include_adx(df , di_period , adx_period):
        adxI = ADXIndicator(df['High'] , df['Low'] , df['Close'] ,20)
        df['pos_directional_indicator'] = adxI.adx_pos()
        df['neg_directional_indicator'] = adxI.adx_neg()
        df['adx'] = adxI.adx()
        #df['+di'] = ta.PLUS_DI(df['high'] , df['ha_low'] , df['close'] , timeperiod= di_period)
        #df['-di'] = ta.MINUS_DI(df['high'] , df['low'] , df['close'] , timeperiod= di_period)
        return df
    
    data_cpy=include_adx(data_cpy,14,30)
    data_cpy['signals'] = 0
    for i in range(1, len(data_cpy)):

        if    ( ((data_cpy['hma'][i] > data_cpy['sma_5'][i] )   and
                 (data_cpy['weight_avg'][i] >= 0.6) and
                    ((data_cpy['macd_signals'].iloc[i]==1) and
                    (data_cpy['atr'].iloc[i]>data_cpy['signal_atr'].iloc[i] and data_cpy['slope_atr'].iloc[i]>0 ) and
                    ( data_cpy['Bull Power'][i]>1.2  )
                ) )  or ((data_cpy['Close'][i]<data_cpy['sma_10'][i] and data_cpy['self_signals'][i]==1 and data_cpy['Bull Power'][i]>1.2 and data_cpy['ema_10'][i]>data_cpy['sma_10'][i] ))
              ):
            # Set signal to 1 for a bullish signal
            data_cpy.signals[i] = 1
    
        # Check if conditions for a bearish signal are met
        if ( ((data_cpy['hma'][i] < data_cpy['sma_5'][i] ) and
                    (
                         (data_cpy['weight_avg'][i] <= 0.38) and
                        (data_cpy['macd_signals'].iloc[i]==-1) and
                    (data_cpy['atr'].iloc[i]<data_cpy['signal_atr'].iloc[i] and data_cpy['slope_atr'].iloc[i]<0  ) and
                    (data_cpy['Bear Power'][i]<-1.2)
    
                       ) ) 
                       or (data_cpy['Close'][i]>data_cpy['sma_10'][i] and data_cpy['self_signals'][i]==-1 and data_cpy['Bear Power'][i]<-1.2 and data_cpy['ema_10'][i]<data_cpy['sma_10'][i] )
                       ):
            # Set signal to -1 for a bearish signal
            data_cpy.signals[i] = -1
    df=data.copy()
    #df=data.copy()
    def identify_support_resistance(df, window=4):
        df['rolling_min'] = df['Low'].rolling(window=window).min()
        df['rolling_max'] = df['High'].rolling(window=window).max()
        df['support'] = df['rolling_min']
        df['resistance'] = df['rolling_max']
        return df

    # Function to generate buy/sell signals based on support and resistance levels
    def generate_signals(df):
        df['signals'] = 0  # Default no position
    
        # Buy when price is near support, Sell when price is near resistance
        df['signals'] = np.where(df['Close'] <= df['support'], 1, df['signals'])  # Buy
        df['signals'] = np.where(df['Close'] >= df['resistance'], -1, df['signals'])  # Sell
        
        return df
    #df = identify_support_resistance(df, window=4)

    # Generate buy/sell signals
    df = generate_signals(df)
    for i in range(len(data_cpy['signals'])):
        if(data_cpy['signals'].iloc[i]==0 and df['signals'].iloc[i]==1):
            data_cpy['signals'].iloc[i]=1
        elif(data_cpy['signals'].iloc[i]==0 and df['signals'].iloc[i]==-1):
            data_cpy['signals'].iloc[i]=-1
    
    data['signals']=data_cpy['signals']
    #data=data.rename(columns={'open':'Open','close':'Close','high':'High','low':'Low'})
                
    return data


### STRATEGY 4

In [13]:
"""
Buy Signal: When a Heikin-Ashi candle changes from red to green, and the volume is significantly higher than the average volume over the last 10 periods.
Sell Signal: When a Heikin-Ashi candle changes from green to red, and the volume is significantly higher than the average volume over the last 10 periods.
"""
def strat4(data):
    data['HA_Close'] = (data['Open'] + data['High'] + data['Low'] + data['Close']) / 4
    data['HA_Open'] = (data['Open'].shift(1) + data['Close'].shift(1)) / 2
    data['HA_High'] = data[['High', 'HA_Open', 'HA_Close']].max(axis=1)
    data['HA_Low'] = data[['Low', 'HA_Open', 'HA_Close']].min(axis=1)
    
    data['Avg_Volume'] = data['Volume'].rolling(window=10).mean()
    data['Volume_Spike'] = data['Volume'] > data['Avg_Volume'] * 1.5
    
    data['signals'] = 0
    data.loc[(data['HA_Close'] > data['HA_Open']) & (data['HA_Close'].shift(1) <= data['HA_Open'].shift(1)) & data['Volume_Spike'], 'signals'] = 1
    data.loc[(data['HA_Close'] < data['HA_Open']) & (data['HA_Close'].shift(1) >= data['HA_Open'].shift(1)) & data['Volume_Spike'], 'signals'] = -1
    
    return data


### RISK MANAGEMENT - ATR BASED STOP-LOSS

In [14]:
def ATR_SL(data,data_cpy,signal, strategynum): # ATR BASED SL takes   df, df+cpy, snum -> modifies df.signals inplace
    type_trade=0  # 1-> long -1->short  0->const
    signals=[]
    current_max=0
    current_min=0
    for i in range(len(data)-1):
        if(type_trade==1):
            if(signal.iloc[i]==0 or signal.iloc[i]==1):
                if(data_cpy['Close'].iloc[i]>atr_take_profit or data_cpy['Close'].iloc[i]<atr_stop_loss):
                    signals.append(-1)
                    type_trade=0
                else:
                    signals.append(0)
            else:
                type_trade=0
                signals.append(-1)

        elif(type_trade==-1):
            if(signal.iloc[i]==0 or signal.iloc[i]==-1):
                if(data_cpy['Close'].iloc[i]<atr_take_profit or data_cpy['Close'].iloc[i]>atr_stop_loss):
                    signals.append(1)
                    type_trade=0
                else:
                    signals.append(0)
            else:
                type_trade=0
                signals.append(1)
        else:
            if(signal.iloc[i]==1):
                signals.append(1)
                type_trade=1
                if(strategynum==1):
                    atr_stop_loss = data_cpy.Close[i] - data_cpy.atr[i]*0.1
                    atr_take_profit=data_cpy.High_s[i] + data_cpy.atr[i]*1.5
                elif(strategynum==2):
                    atr_stop_loss = data_cpy.Low_s[i] - data_cpy.atr[i]*1.5
                    atr_take_profit=data_cpy.High_s[i] + data_cpy.atr[i]*1.9
                elif(strategynum==3):
                    atr_stop_loss = data_cpy.Low_s[i] - data_cpy.atr[i]*2.0
                    atr_take_profit=data_cpy.High_s[i] + data_cpy.atr[i]*2.7
                else:
                    print('What')
                    atr_stop_loss = data_cpy.Low_s[i] - data_cpy.atr[i]*2.5
                    atr_take_profit=data_cpy.High_s[i] + data_cpy.atr[i]*5
            
            elif(signal.iloc[i]==-1):
                signals.append(-1)
                type_trade=-1
                if(strategynum==1):
                    print('SL Hit strat1')
                    atr_stop_loss = data_cpy.Close[i] + data_cpy.atr[i]*0.1
                    atr_take_profit=data_cpy.Low_s[i] - data_cpy.atr[i]*1.5
                elif(strategynum==2):
                    atr_stop_loss = data_cpy.High_s[i] + data_cpy.atr[i]*1.5
                    atr_take_profit=data_cpy.Low_s[i] - data_cpy.atr[i]*1.9
                elif(strategynum==3):
                    atr_stop_loss = data_cpy.High_s[i] + data_cpy.atr[i]*2.0
                    atr_take_profit=data_cpy.Low_s[i] - data_cpy.atr[i]*2.7
                else:
                    print('What error')
                    atr_stop_loss = data_cpy.High_s[i] + data_cpy.atr[i]*2.5
                    atr_take_profit=data_cpy.Low_s[i] - data_cpy.atr[i]*5
            else:
                type_trade=0
                signals.append(0)

    if(type_trade==1):
        signals.append(-1)
    elif(type_trade==-1):
        signals.append(1)
    else:
        signals.append(0)

    data['signals']=signals

    return data

In [15]:
def clean_signals(data): # ensure only 1 active position
    type_trade=0
    for i in range(len(data)-1):
        if(data['signals'].iloc[i]==1 and type_trade==1):
            data['signals'].iloc[i]=0
        elif(data['signals'].iloc[i]==-1 and type_trade==-1):
            data['signals'].iloc[i]=0
        elif((data['signals'].iloc[i]==1 or data['signals'].iloc[i]==-1) and type_trade==0):
            type_trade=data['signals'].iloc[i]
        elif(data['signals'].iloc[i]==1 and type_trade==-1):
            type_trade=0
        elif(data['signals'].iloc[i]==-1 and type_trade==1):
            type_trade=0
    if(type_trade==1):
        data['signals'].iloc[len(data)-1]=-1
    elif(type_trade==-1):
        data['signals'].iloc[len(data)-1]==1
    return data
            
    data["signals"] = signals

    return data


In [16]:
def max_pooling(categories, pool_size):
    pooled_categories = []
    
    # Max pooling over the defined window size
    for i in range(0, len(categories), pool_size):
        segment = categories[i:i + pool_size]
        max_cat = np.max(segment)
        pooled_categories.extend([max_cat] * len(segment))
    
    return pooled_categories


### CLUSTER PREDICTION

In [17]:
def predict_cluster(new_data, centers):
    # Extract and preprocess features from new_data (similar to the training phase)
    features = np.column_stack([
        new_data['VIX'],
        new_data['wick_to_body'],
        new_data['trend'],
        new_data['ATR'],
        new_data['rsi'],
        new_data['volume_spike']
    ])
    features=features.reshape(-1,1)
    
    features = np.array(features, dtype=np.float64)  
    features = np.nan_to_num(features)
    distances = [dtw(features, center) for center in centers]
    predicted_cluster = np.argmin(distances)
    return predicted_cluster

In [18]:
def find_category(data,centers,scaler):

    predicted_cluster = predict_cluster(data, centers)
    # print(f"The new data point belongs to cluster: {predicted_cluster}")
    return predicted_cluster
    
        
    # returns len data // windowlen sized array of category nums 

In [19]:
def calclen(data):
    return min(len(data),50)

In [20]:
def backtest_strategy(tick, sdate, edate):
    data_train, data = get_preprocessed_data_back(tick, sdate, edate)

    if( type(data) == int ):
        if(data == -1):
            print("here")
            return None, None, None, None, "Error please enter date after or equal to 2018-01-01 as 1 year past data is being used"
        

    # fit model using data_train
    # get insight
    data_train, center, scaler = trendpart(data_train,sdate,edate) # data_train now has another colmn for category

    window_len = calclen(data_train) ## implement this

    pred_arr = []
    for i in range(len(data)):
        pred_arr.append(find_category(data.iloc[i],center,scaler))

    data['category'] = 0
    data['category']=pred_arr
    #max pooling
    data['category_pool']=max_pooling(data['category'],pool_size=20)

    # shift category down by window_len to not use future value
    window=7
    data['category_pool']=data['category_pool'].shift(window_len)
    data['rolling_min'] = data['Low'].rolling(window=window).min()
    data['rolling_max'] = data['High'].rolling(window=window).max()
    data['support'] = data['rolling_min']
    data['resistance'] = data['rolling_max']

    data['High_s']=data['Close'].rolling(window=15).max()
    data['Low_s']=data['Close'].rolling(window=15).min()
    
    # now every category corresponds to correct close price
    total_signals = []
    current_win_sig = []
    num_segments = len(data) // window_len
    for seg_no in range(num_segments):
        segment = data.iloc[seg_no * window_len: min(len(data),(seg_no + 1) * window_len)]
        category = segment['category_pool'].iloc[0]  # Get the category of the current segment
        if category == 0:
            segment = strat1(segment)
            segment=clean_signals(segment)
            segment=ATR_SL(segment,segment,segment['signals'],1)
            current_win_sig=segment['signals']
            
        elif category == 1:
            segment = strat2(segment)
            segment=clean_signals(segment)
            segment=ATR_SL(segment,segment,segment['signals'],2)
            current_win_sig=segment['signals']
        elif category == 2:
            segment = strat3(segment)
            segment=clean_signals(segment)
            segment=ATR_SL(segment,segment,segment['signals'],3)
            current_win_sig=segment['signals']
        elif category == 3:
            segment = strat4(segment)
            segment=clean_signals(segment)
            segment=ATR_SL(segment,segment,segment['signals'],4)
            current_win_sig=segment['signals']

        # Ensure the signals are clean (preprocess within the strategy function or here)
        total_signals.extend(current_win_sig)

    # make length of toal_sigmals and data equal by appending 0s in total_signlas
    if len(total_signals) < len(data):
        total_signals.extend([0] * (len(data) - len(total_signals)))

    # Assign the signals to the data
    data["signals"] = total_signals

    data['low']=data['Low']
    data['high']=data['High']
    data['close']=data['Close']
    data['open']=data['Open']
    data['volume']=data['Volume']
    data['datetime'] = data.index
    
    Testing_signals = TradingStrategy_Compounding(data)
    to_submit_comp, tradewise, daily, graph, metricstr = Testing_signals.compounding()
    
    
    # returnables = backtest(data, 1, 1, data["FinalSignals"])  # Adjust the parameters as necessary
    data['category_pool'] = data['category_pool'].fillna(0)
    catarr = data['category_pool'].to_numpy()

    tradewise['cluster'] = [catarr[i] for i in tradewise['entry']]
    
    daily['cluster'] = data['category_pool'].to_numpy()
    return data, daily, tradewise, graph, metricstr
    # df, daily, tradewise, graph, metricstr
    

In [21]:
# data, dailydf, tradedf, f, metricstr=backtest_strategy("MSFT","2017-01-01","2018-01-01")

In [22]:
def clean_signals_fwd(data):
    type_trade=0
    for i in range(len(data)):
        if(data['signals'].iloc[i]==1 and type_trade==1):
            data['signals'].iloc[i]=0
        elif(data['signals'].iloc[i]==-1 and type_trade==-1):
            data['signals'].iloc[i]=0
        elif((data['signals'].iloc[i]==1 or data['signals'].iloc[i]==-1) and type_trade==0):
            type_trade=data['signals'].iloc[i]
        elif(data['signals'].iloc[i]==1 and type_trade==-1):
            type_trade=0
        elif(data['signals'].iloc[i]==-1 and type_trade==1):
            type_trade=0
    data['signals']=data['signals']
    return data

### FORWARD TESTING

In [23]:
import random
def forwardtest_strategy(data):
    
    data['SMA5'] = data['Close'].rolling(window=5).mean()
    data['SMA20'] = data['Close'].rolling(window=20).mean()
    
    signals = np.where((data['SMA5'] > data['SMA20']) & (data['Close']), 1, -1)
    data['signals'] = 0
    data['signals'][5:] = signals[5:]

    # signalsarray=random.shuffle([-1]*30+[1]*(len(data)-30))
    # data['signal'] = signalsarray
    
    
    data = clean_signals_fwd(data)
    
    returnables = backtest(data, 0.05, 0.001, data.FinalSignals)
    return returnables


 ### GRADIO CODE ---

In [24]:
def plot_historical_results(df, daily, tradewise, graph):
    #print("from plothistorical", df)
    fig1buy = pd.DataFrame()
    fig1sell = pd.DataFrame()
    print('df\n',df)
    daily.drop('Close',axis=1,inplace=True)
    print('daily',daily)
    daily['Close'] = df.Close.to_numpy()
    print('daily',daily)
    
    for _, row in daily.iterrows():
        if row['signals'] > 0:
            print("plotting buy")
            new_row = pd.DataFrame({"x": [row['datetime']], "y": [row['Close']], "type": ["buy"]})
            fig1buy = pd.concat([fig1buy, new_row], ignore_index=True)
        elif row['signals'] < 0:
            print("plotting sell")
            new_row = pd.DataFrame({"x": [row['datetime']], "y": [row['Close']], "type": ["sell"]})
            fig1sell = pd.concat([fig1sell, new_row], ignore_index=True)

    #  'g^', markersize=10
    #  'rv', markersize=10
    
    
    df1 = pd.DataFrame({"x": df.index, "y":df.Close, "type":"line"})
    
    df2 = pd.DataFrame({"x": df.index, "y":daily['portfolio value']})
    
    fig1 = go.Figure()
    fig2=go.Figure()

    fig1.add_trace(go.Scatter(x=df1["x"], y=df1["y"], mode='lines', name='ClosePrice'))
    fig2.add_trace(go.Scatter(x=df2["x"], y=df2["y"], mode='lines', name='Portfolio'))

    print("plotting fig1buysell  ",fig1buy, fig1sell)
    
    fig1.add_trace(go.Scatter(x=fig1buy["x"], y=fig1buy["y"], mode='markers', marker=dict(color='green', symbol='triangle-up', size=8), name='BUY'))
    fig1.add_trace(go.Scatter(x=fig1sell["x"], y=fig1sell["y"], mode='markers', marker=dict(color='red', symbol='triangle-down', size=8), name='SELL'))

    # fig1_json = pio.to_json(fig1, validate=False)
    # fig2_json = pio.to_json(fig2, validate=False)
    
    return fig1, fig2

def plot_live_results(df, daily, tradewise, graph):

    daily.drop('Close',axis=1,inplace=True)
    daily['Close'] = df.Close.to_numpy()
    fig1buy = pd.DataFrame()
    fig1sell = pd.DataFrame()
    
    for _, row in daily.iterrows():
        if row['signals'] > 0:
            new_row = pd.DataFrame({"x": [row['datetime']], "y": [row['Close']], "type": ["buy"]})
            print("rowclose",row['Close'])
            fig1buy = pd.concat([fig1buy, new_row], ignore_index=True)
            print(f"appending {_} to buy")
        elif row['signals'] < 0:
            new_row = pd.DataFrame({"x": [row['datetime']], "y": [row['Close']], "type": ["sell"]})
            fig1sell = pd.concat([fig1sell, new_row], ignore_index=True)
            print(f"appending {_} to sell")

    #  'g^', markersize=10
    #  'rv', markersize=10
    
    
    df1 = pd.DataFrame({"x": df.index, "y":df.Close, "type":"line"})
    
    df2 = pd.DataFrame({"x": df.index, "y":daily['portfolio value']})
    
    fig1 = go.Figure()
    fig2=go.Figure()

    fig1.add_trace(go.Scatter(x=df1["x"], y=df1["y"], mode='lines', name='ClosePrice'))
    fig2.add_trace(go.Scatter(x=df2["x"], y=df2["y"], mode='lines', name='Portfolio'))

    try:
        fig1.add_trace(go.Scatter(x=fig1buy["x"], y=fig1buy["y"], mode='markers', marker=dict(color='green', symbol='triangle-up', size=8), name='BUY'))
    except Exception as e:
        print(e, end="")
        print("in buy")
    try:
        fig1.add_trace(go.Scatter(x=fig1sell["x"], y=fig1sell["y"], mode='markers', marker=dict(color='red', symbol='triangle-down', size=8), name='SELL'))
    except Exception as e:
        print(e, end="")
        print("in sell")
    
    print(fig1buy)
    print(fig1sell)

    
    # fig1.add_trace(go.Scatter(x=fig1buy["x"], y=fig1buy["y"], mode='markers', marker=dict(color='green', symbol='triangle-up', size=8), name='BUY'))
    # fig1.add_trace(go.Scatter(x=fig1sell["x"], y=fig1sell["y"], mode='markers', marker=dict(color='red', symbol='triangle-down', size=8), name='SELL'))
    
    # fig1_json = pio.to_json(fig1, validate=False)
    # fig2_json = pio.to_json(fig2, validate=False)
    
    return fig1, fig2
    

def historical_trading(ticker, start_date, end_date):
    # (Keep the existing implementation)
    df, daily, tradewise, graph, metricstr = backtest_strategy(ticker, start_date, end_date)
    try:
        if (df)==None:
            return daily, tradewise, metricstr, None, None
    except Exception as e:
        print(e)
    daily['Close'] = df['close']
    fig1, fig2 = plot_historical_results(df, daily, tradewise, graph)
    return daily, tradewise, metricstr, fig1, fig2

def live_trading(ticker, intv):
    global should_stop
    should_stop = False  # Reset the stop flag
    datatrain = yf.download(ticker, "2022-01-01", "2023-01-01")
    vix_symbol = "^VIX"
    datatrain.columns = datatrain.columns.str.capitalize()
    #data.columns = data.columns.str.capitalize()
    #vix_data_t = yf.download(vix_symbol, import_date, sdate)
    vix_data = yf.download(vix_symbol, "2022-01-01", "2023-01-01")
    #data_train['VIX']=vix_data['Close']
    datatrain['VIX']=vix_data['Close']
    datatrain=gen_newdata(datatrain) # calculate the ichimoku metrics similar to historical for finding clustering params
    datatrain = datatrain.dropna()
    # return data_train, data
    window=7
    datatrain['rolling_min'] = datatrain['Low'].rolling(window=window).min()
    datatrain['rolling_max'] = datatrain['High'].rolling(window=window).max()
    datatrain['support'] = datatrain['rolling_min']
    datatrain['resistance'] = datatrain['rolling_max']
    datatrain['High_s']=datatrain['Close'].rolling(window=15).max()
    datatrain['Low_s']=datatrain['Close'].rolling(window=15).min()
    
    
    datatrain, center, scaler = trendpart(datatrain,"2022-01-01","2023-01-01")# data_train now has another colmn for category
    window_len = calclen(datatrain)
    
    
    
    if(1==1):
    # try:
        tmp= 0
        while not should_stop:
            tmp+=1
            print("in while loop", tmp)
            data = yf.download(ticker, str(datetime.datetime.today()).split()[0], str(datetime.datetime.today()+100*datetime.timedelta(days=1)).split()[0], interval=intv)
            vix_data = yf.download(vix_symbol, str(datetime.datetime.today()).split()[0], str(datetime.datetime.today()+100*datetime.timedelta(days=1)).split()[0], interval=intv)
            data['VIX']=vix_data['Close']
            data=gen_newdata(data)
            # data.columns = data.columns.str.lower()
            data['rolling_min'] = data['Low'].rolling(window=window).min()
            data['rolling_max'] = data['High'].rolling(window=window).max()
            data['support'] = data['rolling_min']
            data['resistance'] = data['rolling_max']
            data['High_s']=data['Close'].rolling(window=15).max()
            data['Low_s']=data['Close'].rolling(window=15).min()
            if(len(data)<10):
                print('Looping As data less than 10', data)
                continue          
            pred_arr = []
            for i in range(len(data)):
                pred_arr.append(find_category(data.iloc[i],center,scaler))
        
            data['category'] = 0
            data['category']=pred_arr
            #max pooling
            data['category_pool']=max_pooling(data['category'],pool_size=20)
        
            # shift category down by window_len to not use future value
            data['category_pool']=data['category_pool'].shift(window_len)
            window=7
            data['rolling_min'] = data['Low'].rolling(window=window).min()
            data['rolling_max'] = data['High'].rolling(window=window).max()
            data['support'] = data['rolling_min']
            data['resistance'] = data['rolling_max']
            
            # now every category corresponds to correct close price
            total_signals = []
            current_win_sig = []
            num_segments = len(data) // window_len
            for seg_no in range(num_segments):
                segment = data.iloc[seg_no * window_len: min(len(data),(seg_no + 1) * window_len)]
                category = segment['category_pool'].iloc[0]  # Get the category of the current segment
                if category == 0:
                    segment = strat1(segment)
                    segment=clean_signals(segment)
                    segment=ATR_SL(segment,segment,segment['signals'],1)
                    current_win_sig=segment['signals']
                    
                elif category == 1:
                    segment = strat2(segment)
                    segment=clean_signals(segment)
                    segment=ATR_SL(segment,segment,segment['signals'],2)
                    current_win_sig=segment['signals']
                elif category == 2:
                    segment = strat3(segment)
                    segment=clean_signals(segment)
                    segment=ATR_SL(segment,segment,segment['signals'],3)
                    current_win_sig=segment['signals']
                elif category == 3:
                    segment = strat4(segment)
                    segment=clean_signals(segment)
                    segment=ATR_SL(segment,segment,segment['signals'],4)
                    current_win_sig=segment['signals']
        
                # Ensure the signals are clean (preprocess within the strategy function or here)
                total_signals.extend(current_win_sig)
        
            # make length of toal_sigmals and data equal by appending 0s in total_signlas
            if len(total_signals) < len(data):
                total_signals.extend([0] * (len(data) - len(total_signals)))
    
            data["signals"] = total_signals
    
    
            data['low']=data['Low']
            data['high']=data['High']
            data['close']=data['Close']
            data['open']=data['Open']
            data['volume']=data['Volume']
            data['datetime'] = data.index
            
            Testing_signals = TradingStrategy_Compounding(data)
            to_submit_comp, tradewise, daily, graph, metricstr = Testing_signals.compounding()
        
            
            # returnables = backtest(data, 1, 1, data["FinalSignals"])  # Adjust the parameters as necessary
        
            #return data, daily, tradewise, graph, metricstr
    
            #df_live, daily, tradewise, graph, metricstr = forwardtest_strategy(data)
            fig1, fig2 = plot_live_results(data, daily, tradewise, graph)
    
            yield fig1, tradewise, metricstr, fig2
            
            time.sleep(3) # refresh rate
        
    # except Exception as e:
    #     print(f"An error occurred: {e}")
    #     print(data)
    # finally:
    #     print("in the finally block")
    #     pass
    #     # yield None, None, f"Trading stopped. Final metrics:", None

def stop_trading():
    global should_stop
    should_stop = True
    print("Trading stop signal sent.")

with gr.Blocks() as demo:
    gr.Markdown("# Trading Bot Interface")
    
    with gr.Tabs():
        with gr.TabItem("Historical Trading"):
            with gr.Row():
                ticker_hist = gr.Textbox(label="Enter Ticker Symbol")
                start_date = gr.Textbox(label="Start Date (YYYY-MM-DD)")
                end_date = gr.Textbox(label="End Date (YYYY-MM-DD)")
            submit_hist = gr.Button("Run Historical Trading")
            
            daily = gr.Dataframe(label="Daily Data")
            tradewise = gr.Dataframe(label="Trade-wise Data")
            metricstr = gr.Textbox(label="Metrics")
# PLOT1 ______________________
            plot1 = gr.Plot()
# PLOT2________________________
            plot2 = gr.Plot()

        with gr.TabItem("Live Trading"):
            with gr.Row():
                ticker_live = gr.Textbox(label="Enter Ticker Symbol", scale=3)
                options = ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d']
                intv = gr.Dropdown(choices=options, label="Select Time Interval", value=options[0], scale=1)
                
            with gr.Row():
                live_submit = gr.Button("Start Live Trading")
                stop_button = gr.Button("Stop Trading")
            live_plot_close = gr.Plot(label="Live Trading Plot")
            live_tradewise = gr.Dataframe(label="Trade-wise Data")
            portfolio_plot = gr.Plot(label="Portfolio")


            live_metrics = gr.Textbox(label="Final Metrics")

    
    # Link the buttons to functions
    submit_hist.click(
        historical_trading,
        inputs=[ticker_hist, start_date, end_date], 
        outputs=[daily, tradewise, metricstr, plot1, plot2]
    )
    live_submit.click(
        live_trading, 
        inputs=[ticker_live, intv], 
        outputs=[live_plot_close, live_tradewise, live_metrics, portfolio_plot]
    )
    stop_button.click(stop_trading)

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [25]:
%matplotlib inline